# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
city_vac_df = pd.read_csv("../output_data/city_data1.csv")
city_vac_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,benito juarez,75,MX,1587428189,34,19.39,-99.17,80.60,17.22
1,1,hobart,40,AU,1587427898,66,-42.88,147.33,55.00,9.17
2,2,rikitea,24,PF,1587428092,71,-23.12,-134.97,77.13,9.42
3,3,santiago,69,CL,1587428102,23,-33.46,-70.65,77.00,8.05
4,4,victoria,20,HK,1587428114,74,22.29,114.16,81.00,4.70


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
vac_locations = city_vac_df[["Lat", "Lng"]]
vac_weight    = city_vac_df['Humidity'].astype(int)

# Plot Heatmap
gmaps_fig = gmaps.figure()

# Create heat layer
gmaps_heat = gmaps.heatmap_layer(vac_locations, weights=vac_weight,dissipating=False, max_intensity=10,point_radius=1) 
                                   
# Add layer
gmaps_fig.add_layer(gmaps_heat)
gmaps_fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display Map